<a href="https://colab.research.google.com/github/mlopezd01/notebook10/blob/main/Notebook_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Programa Ingenia+ Data Science

Recordemos que un proyecto de data science tiene varias etapas:

1. Recolección de Datos
2. Exploración y Procesamiento de los datos
3. Modelado
4. Puesta en Producción

En clases anteriores, trabajamos con el dataset `StudentPerformace`. Pudimos observar el tipo de datos que teniamos y le hiciemos algunas transformaciones.

Llego la hora de utilizar los conceptos de aprendizaje supervisado para hacer predicciones sobre alguna variable en el dataset.

Para poder ajustar modelos de Machine Learning, la herramienta más sencilla y más utilizada es `scikit-learn`. Esta librería es muy utilizada ya que contiene funciones desarrolladas para la mayoría de los modelos de machine learning. Además `Scikit-learn` tienen una amplia y muy buena [documentación](https://scikit-learn.org/stable/).

<font size=5> 🚀 👩🏽‍💻 Machine Learning: Aprendizaje Supervisado 📣</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [2]:
import os

In [3]:
os.getcwd()

'/content'

Primero, vamos a leer los datos limpios y procesados.

In [6]:
# Leemos nuevamente los datos de los estudiantes
students = pd.read_csv('/content/estudiantes_limpio.csv')

In [7]:
students.head()

,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,english score,female,male,associate's degree,bachelor's degree,high school,master's degree,some college,some high school
0,1,1,1,72.0,72,74,73.0,1,0,0,1,0,0,0,0
1,2,1,0,69.0,90,88,89.0,1,0,0,0,0,0,1,0
2,1,1,1,90.0,95,93,94.0,1,0,0,0,0,1,0,0
3,0,0,1,47.0,57,44,50.5,0,1,1,0,0,0,0,0
4,2,1,1,76.0,78,75,76.5,0,1,0,0,0,0,1,0


Antes que nada, chequeemos nuevamente que eliminamos todos los valores faltantes.

In [8]:
students.isnull().sum()

,0
race/ethnicity,0
lunch,0
test preparation course,0
math score,0
reading score,0
writing score,0
english score,0
female,0
male,0
associate's degree,0


Ahora tenemos que decidir que variable o variables vamos a predecir. Esto quiere decir que tenemos que elegir cual va a ser la variable que al predecirla conteste la pregunta que nos hicimos de los datos.

Lo que queremos saber es:

1. ¿Qué puntaje final obtendra un estudiante?
2. Dado un estudiante, ¿aprobara o no el examen?

**_¿Qué tipo de problemas estamos tratando de resolver?_**

Para contestar estas preguntas vamos a obtener dos nuevas variables:

1. El puntaje final: Lo obtendremos promediando los tres puntajes (matematica, lectura y escritura)
2. Aprobado: Una etiqueta que me permita saber si cada estudiante del dataset aprobo el examen (puntaje final >= 60) o no lo aprobo (puntaje final < 60).

In [9]:
students['puntaje_final'] = (students['math score'] + students['reading score'] + students['writing score']) / 3

In [11]:
students.head()

,race/ethnicity,lunch,test preparation course,math score,reading score,writing score,english score,female,male,associate's degree,bachelor's degree,high school,master's degree,some college,some high school,puntaje_final
0,1,1,1,72.0,72,74,73.0,1,0,0,1,0,0,0,0,72.666667
1,2,1,0,69.0,90,88,89.0,1,0,0,0,0,0,1,0,82.333333
2,1,1,1,90.0,95,93,94.0,1,0,0,0,0,1,0,0,92.666667
3,0,0,1,47.0,57,44,50.5,0,1,1,0,0,0,0,0,49.333333
4,2,1,1,76.0,78,75,76.5,0,1,0,0,0,0,1,0,76.333333


## 📈 Regresión

Comenzaremos contestando la pregunta 1. Este problema es un problema de regresión (_¿Por qué?_)

porque utiliza variables numericas.

### Seleccion de target (y) y variables (X)

Lo primero que haremos, es seleccionar las variables o features que deseemos usar como variable independientes (aquellas que van a explicar mi predicción) y la variable a predecir.

Usaremos todas las variables menos aquellas que fueron usadas para calcular el puntaje final. Debemos también asegurarnos de no incluir las variables a predecir. Como variable a predecir usaremos el puntaje final.

In [12]:
# Elimino aquellas variables que no quiero incluir en el modelo y las guardo en x.
x = students.drop(['math score', 'reading score', 'writing score', 'english score', 'puntaje_final'],
                  axis=1)

In [13]:
x.head()

,race/ethnicity,lunch,test preparation course,female,male,associate's degree,bachelor's degree,high school,master's degree,some college,some high school
0,1,1,1,1,0,0,1,0,0,0,0
1,2,1,0,1,0,0,0,0,0,1,0
2,1,1,1,1,0,0,0,0,1,0,0
3,0,0,1,0,1,1,0,0,0,0,0
4,2,1,1,0,1,0,0,0,0,1,0


In [14]:
lista_atributos = x.columns

In [15]:
# Ahora selecciono las etiquetas y las guardo en y.
y = students['puntaje_final']

In [17]:
#x son los nuevos atributos, y la variable a pr4edecir
x, y = np.array(x), np.array(y)

In [18]:
x

array([[1, 1, 1, ..., 0, 0, 0],
       [2, 1, 0, ..., 0, 1, 0],
       [1, 1, 1, ..., 1, 0, 0],
       ...,
       [2, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 1, 0],
       [3, 0, 1, ..., 0, 1, 0]])

In [19]:
y[:10]

array([72.66666667, 82.33333333, 92.66666667, 49.33333333, 76.33333333,
       77.33333333, 91.66666667, 40.66666667, 65.        , 49.33333333])

### Separando Train / Test

Ahora tenemos que separar el dataset en conjuntos de entrenamiento (X_train, y_train) y de testeo (X_test, y_test) usando la función train_test_split de scikit-learn.

¿Por qué separamos el dataset? Porque es necesario entrenar el modelo con un set de datos. El modelo no puede ver los datos de evaluación cuando se esta entrenando!

La forma en que se dividen los datos normalmente es 80% entrenamiento -20% evaluación)

In [20]:
# Importamos la librearia para separar el dataset.
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    random_state=42)

In [22]:
x.shape

(1000, 11)

In [23]:
X_train.shape

(800, 11)

In [24]:
X_test.shape

(200, 11)

### Random Forest

Recordemos que Random Forest es un algoritmo de ensamble. Hay dos tecnicas para esto: Bagging y Boosting.

El objetivo de bagging es entrenar distintos modelos, donde cada uno vea distintas porciones del set de entrenamiento. Random Forest, además de aplicar Bagging, también selecciona features al azar, de esa manera descorrelaciona aún más los distintos modelos de árbol creados.

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
from sklearn.metrics import mean_absolute_error

In [32]:
# Inicializo el modelo
regresor = RandomForestRegressor(criterion=' ', random_state=42)

In [33]:
# Entreno el modelo
regresor.fit(X_train, y_train);

In [ ]:
regresor.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mae',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

Ahora evaluaremos el modelo (Profundizaremos este concepto más adelante). Primero, hacemos predicciones para el set de evaluación y luego lo comparamos con los valores reales.

In [34]:
# Predigo los valores para el set de testeo
y_pred = regresor.predict(X_test)

In [35]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred)

11.390941666666668

### Support Vector Machine

Recordemos que el SVM es un algortimo que establece un hiperplano que separa los puntos maximizando el margen. A su vez, una de sus ventajas es la aplicación de Kernels que me permite ajustar mejor el modelo.


In [36]:
from sklearn.svm import SVR

In [37]:
# Inicializo el modelo
regresor_svr = SVR(C=1.0, epsilon=0.2)

In [38]:
# Entreno el modelo
regresor_svr.fit(X_train, y_train);

In [40]:
regresor_svr.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.2,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

Ahora evaluaremos el modelo (Profundizaremos este concepto más adelante). Primero, hacemos predicciones para el set de evaluación y luego lo comparamos con los valores reales.

In [41]:
# Predigo los valores para el set de testeo
y_pred_svr = regresor_svr.predict(X_test)

In [42]:
# Calculo el error medio absoluto
mean_absolute_error(y_test, y_pred_svr)

10.610220119303728